In [1]:
# Import files
import moviepy.editor as mp 
import wave
import numpy as np

In [2]:
word_size = 16 # Size of the number in byte format
secret_key = 567 # Secret key already known by the program 

### Encryption

In [3]:
# Load video files and change it into the wav format
video = mp.VideoFileClip("Small.mp4") 
audio = video.audio
audio.write_audiofile("Audio.wav")
audio = wave.open("Audio.wav", mode='rb')

chunk:   0%|                                                                         | 0/775 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in Audio.wav


MoviePy - Done.


In [4]:
# Load bytes from the audio file
frame_data = audio.readframes(audio.getnframes())
frame_bytes = bytearray(list(frame_data))

In [5]:
# Frame data to be written first element is number of frame object
frame_data = [4,165,175,200,1000]
number_element = frame_data[0]

In [6]:
# Convert the frame into bit data of word size and then store it in the audio file
frame_data = ''.join(list(map(lambda number: '{0:0{width}b}'.format(number,width=word_size),frame_data)))
frame_data = list(map(int,frame_data))
frame_bytes_length = len(frame_bytes)

In [7]:
number_random_position = np.random.RandomState(seed=secret_key).choice(frame_bytes_length,word_size) # Get word_size random positions

In [8]:
# Peform steganography encrpt the number of frames in random places
for i, bit in enumerate(frame_data[:word_size]):
    frame_bytes[number_random_position[i]] = (frame_bytes[number_random_position[i]] & 254) | bit

In [9]:
# Remove encoded values from frame data and get random location to write data
frame_data = frame_data[word_size:]
random_bit_location = np.random.RandomState(seed=secret_key*number_element).choice(frame_bytes_length,word_size*(number_element+1))
random_bit_location = list(set(random_bit_location) - set(number_random_position))[:word_size*number_element]

In [10]:
# Peform steganography and save the data in the encrypted format
for i, bit in enumerate(frame_data):
    frame_bytes[random_bit_location[i]] = (frame_bytes[random_bit_location[i]] & 254) | bit

In [11]:
# Generate modified frames
modified_frames = bytes(frame_bytes)

In [12]:
# Store the final audio file
final_audio = wave.open("Encrypted.wav","wb")
final_audio.setparams(audio.getparams())
final_audio.writeframes(modified_frames)

### Decryption

In [13]:
# Load the final audio file and extract frames from it
audio = wave.open("Encrypted.wav", mode="rb")
frame_data = audio.readframes(audio.getnframes())
frame_bytes = bytearray(list(frame_data))

In [14]:
frame_bytes_length = len(frame_bytes) # Get number of frames
number_random_position = np.random.RandomState(seed=secret_key).choice(frame_bytes_length,word_size) # Get word_size random positions

In [15]:
# Extract bits from the message and get number of frames from data
extracted = [frame_bytes[number_random_position[i]] & 1 for i in range(word_size)]
number_element = int(str("".join(list(map(str,extracted)))),2)

In [16]:
# Get random location to read data
random_bit_location = np.random.RandomState(seed=secret_key*number_element).choice(frame_bytes_length,word_size*(number_element+1))
random_bit_location = list(set(random_bit_location) - set(number_random_position))[:word_size*number_element]

In [17]:
# Extract frame data
finalExtraction = [] # Final extracted frame data
for i in range(0,(number_element*word_size)):
    finalExtraction.append(frame_bytes[random_bit_location[i]] & 1)

In [18]:
# Extract bits and club them together to form word and then convert it to numbers
frame_list = list(np.array(finalExtraction).reshape(number_element,word_size))
number = list(map(lambda x: int("".join(list(map(str,x))),2),frame_list))

In [19]:
print(number)

[165, 175, 200, 1000]
